# Gene-level RNA-seq differential expression analysis from aligned data
[Paolo Sonego](paolo.sonego@fmach.it)<sup>*</sup>

<sup>*</sup>Computational Biology Unit,Fondazione Edmund Mach, <br/>
Via Edmund Mach 1, 38010, S. Michele All'Adige, Trento,  Italy

## Introduction
In this practical we will show a standard workflow for the analysis of RNA-seq data. It is not intended as THE Definitive Guide in the analysis of this kind of data; we will cover one of the many existing workflows to analyse RNA-Seq data for a common task, i.e. Differential Expression Analysis.

## Analysis overview
1. Retrieving RNA-seq reads from public repositories
2. Pre-processing (Quality Control, trimming)
3. Alignment and Counting
4. Exploratory Data Analysis
5. Filtering and Normalization
6. Differential Expression Analysis
7. Gene Ontology Enrichment

## Experiment info
As a running example we use a dataset which is available from both the Short Read archive (SRA Study ID: SRP193991 - raw data) and from GEO (GEO Experiment ID: GSE130386 - processed data). The authors investigate the Muscat flavour of three Vitis vinifera genotypes with different aromatic strength by integrating transcript and terpene content metabolite (from which the flavour originates) profiles. 
> A total of 27 fruit samples from three biological replicates were sequenced on Illumina HiSeq2000 at three stages, corresponding to the veraison; berries had intermediate Brix value and were harvest-ripe. 

For more information about this dataset please refer to the original publication:[Transcriptome profiles of three Muscat table grape cultivars to dissect the mechanism of terpene biosynthesis](https://www.nature.com/articles/s41597-019-0101-y). <a href="#ref1">[1]</a>

| Design Type(s) | transcription profiling design • gene expression analysis objective |
|--------------------------|---------------------------------------------------------------------|
| Measurement Type(s) | transcription profiling assay |
| Technology Type(s) | RNA sequencing |
| Factor Type(s) | cultivar • biological replicate • developmental stage |
| Sample Characteristic(s) | Vitis vinifera • berry |

The tools and R packages that we will be using during the practical are listed below and the data used for the analysis can be found at /home/share/RNASeqPractical/.

# Setup

In [1]:
## Setup - loading libraries for the analysis
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(plotly))
suppressPackageStartupMessages(library(RColorBrewer))
suppressPackageStartupMessages(library(Rsubread))
suppressPackageStartupMessages(library(tximport))
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(edgeR))
suppressPackageStartupMessages(library(limma))
suppressPackageStartupMessages(library(goseq))
suppressPackageStartupMessages(library(GO.db))

## Setup - setting up paths and options
shared="/home/share/RNASeqPractical"
data="/home/share/RNASeqPractical/data"
tools="/home/share/RNASeqPractical/tools"
references="/home/share/RNASeqPractical/references"
options(repr.plot.width = 15, repr.plot.height = 15) # set plot size

# Retrieving RNA-seq reads from public repositories

## Download raw data (sra/fastq) from NCBI's Sequence Read Archive (SRA)
SRA is one of the primary repositories for next-generation sequencing data. Submissions to SRA can be complex, with multiple samples from a given submission, or multiple sequencing runs associated with a given sample. As a result, a given SRA submission “envelope” can be associated with database records at several levels, each with corresponding identifiers and nomenclature. Valid levels include: the study (prefixed with SRP), the experiment (prefixed with SRX), the sample (prefixed with SRS), and the run (prefixed with SRR). Mapping identifiers between the different parts of an SRA submission is important when you need to access meta-data from different parts of a submission or you need to aggregate data across different levels of a submission (e.g. sequencing runs from the same biological sample)<a href="#ref2">[2]</a>

**DO NOT RUN!!!**
```{r}
suppressPackageStartupMessages(library(SRAdb))
suppressPackageStartupMessages(library(DBI))
if(!file.exists('SRAmetadb.sqlite')) sra_dbname <<- getSRAdbFile()
sra_dbname <- 'SRAmetadb.sqlite'        
con <- dbConnect(dbDriver("SQLite"), sra_dbname)
listSRAfile('SRP193991', con)
getSRAfile('SRP193991', con, fileType = 'fastq', srcType = 'ftp', makeDirectory = TRUE) 
```
or

**DO NOT RUN!!!**
```sh
pysradb download --srp SRP193991
```

# Quality Control

### Activity
- Use FastQC  <a href="#ref3">[3]</a> to generate Quality Control reports for the raw data (fastq files).
- Check the fastqc report on your browser (notebook session)

#### Hint

- Open a terminal and browse the RNASeqPractical directory (`/home/share/RNASeqPractical/data`) looking for the fastq raw data
- From the command line take advantage of the below template to select few samples to check.

```sh
cd $HOME/day_2_rnaseq/
tools="/home/share/RNASeqPractical/tools"
fastq_in="<fastq raw path here>"
mkdir -p $HOME/day_2_rnaseq/QC/raw
fastqc_out=$HOME/day_2_rnaseq/QC/raw
for f in `basename -s .fastq.gz $fastq_in/*.fastq.gz`;
do
  $tools/FastQC/fastqc $fastq_in/$f.fastq.gz -t 1 --extract -o $fastqc_out
done
```
- Use the below base code to inspect the result on your notebook

```{r}
qc_list <- list.files(path="QC/raw",pattern="*.html", full.names=T)
IRdisplay::display_html(paste0(capture.output(print(paste0("<a href=", qc_list,">",qc_list,"</a>"), type = 'html')), collapse=""))
#IRdisplay::display_html('<iframe src="QC/raw/SRR8979775_fastqc/fastqc_report.html" width=1000, height=1000></iframe> ')
```

# Trimming
Raw data in the fastq format was processed and adaptor and low-quality sequence reads were removed from the data sets using trimmomatic <a href="#ref4">[4]</a>. To run the full samples from this dataset would take several hours.

```sh
cd $HOME/day_2_rnaseq/
tools="/home/share/RNASeqPractical/tools"
fastq_in="/home/share/RNASeqPractical/data/fastq/raw"
mkdir -p $HOME/day_2_rnaseq/fastq/trimmed
fastq_out=$HOME/day_2_rnaseq/fastq/trimmed
for f in `basename -s .fastq.gz $fastq_in/*.fastq.gz`;
do
  java -Xmx2096m -jar $tools/Trimmomatic-0.39/trimmomatic-0.39.jar SE -phred33 -threads 8 \
  $fastq_in/$f.fastq.gz \
  $fastq_out/${f}_trim.fastq.gz \
  ILLUMINACLIP:/home/share/RNASeqPractical/tools/Trimmomatic-0.39/adapters/TruSeq3-PE-2.fa:2:30:10
done
```

### Activity
- Open a Terminal, run the trimming software, run FastQC on the result and compare the  report for both pre- and post- trimming fastq files.

### hint
**This step requires a lot of time so try it with one sample** and check the FastQC reports for both raw and trimmed data.

# Alignment
## Build the index
The read sequences are stored in (compressed) FASTQ files. These files must be aligned to the Vitis vinifera genome and counted into annotated genes. The first step in performing the alignment is to build an index. In order to build an index you need to have the fasta file (.fa), which can be downloaded from [Vitis vinifera (12x) genome plants ensembl](http://plants.ensembl.org/Vitis_vinifera/Info/Index) or [CRIBI](http://genomes.cribi.unipd.it/DATA) or [INRA](https://urgi.versailles.inra.fr/download/vitis).
This operation can be done from either the command line:
```sh
cd $HOME/day_2_rnaseq/
tools="/home/share/RNASeqPractical/tools"
references="/home/share/RNASeqPractical/references"
mkdir -p cd $HOME/day_2_rnaseq/subread_idx
$tools/subread-2.0.0-Linux-x86_64/bin/subread-buildindex -B -o subread_idx/Vitis_vinifera.12X.dna.toplevel.subread $references/ensemblgenomes_Vitis_vinifera_release-43/fasta/dna/Vitis_vinifera.12X.dna.toplevel.fa
```
or in R using the `Rsubread` package:
```{r}
references="/home/share/RNASeqPractical/references"
dir.create("subread_idx", showWarnings = T)
buildindex(basename="subread_idx/Vitis_vinifera.12X.dna.toplevel.subread",
reference=paste0(references,"/ensemblgenomes_Vitis_vinifera_release-43/fasta/dna/Vitis_vinifera.12X.dna.toplevel.fa"))
```

## Aligning reads to reference genome

Now that we have generated our index, we can align our reads using the align command using the default mapping parameters. To run the full samples from this dataset would take several hours. The BAM files are saved in the working directory.
To see how many parameters you can change try the `args` function:```{r} args(align) ``` or better take a look at the [documentation](http://subread.sourceforge.net/).The alignement can be done from either the command line:

```sh
cd $HOME/day_2_rnaseq/
subread_idx="$HOME/day_2_rnaseq/subread_idx"
tools="/home/share/RNASeqPractical/tools/subread-2.0.0-Linux-x86_64/bin"
fastq="/home/share/RNASeqPractical/data/fastq/raw"
#fastq="/home/share/RNASeqPractical/data/fastq/trimmed"
mkdir -p $HOME/day_2_rnaseq/bam/raw
#mkdir -p $HOME/day_2_rnaseq/bam/trimmed
bam=$HOME/day_2_rnaseq/bam/raw
#$HOME/day_2_rnaseq/bam/trimmed
$tools/subread-align -T 1 -t 0 -i $subread_idx/Vitis_vinifera.12X.dna.toplevel.subread \
  -r $fastq/SRR8979775.fastq.gz -o $bam/SRR8979775.bam
```

or R:

```{r}
subread_idx='subread_idx'
fastq="/home/share/RNASeqPractical/data/fastq/raw"
dir.create('bam/raw', recursive = TRUE)
bam="bam/raw"
files = list.files(fastq, pattern="fastq.gz",full.names=FALSE)
f = 1
align(index=paste0(subread_idx,"/Vitis_vinifera.12X.dna.toplevel.subread"), 
      readfile1=list.files(fastq, pattern="fastq.gz",full.names=TRUE)[f], 
      output_file=paste0(bam, "/", tools::file_path_sans_ext(files[f], compression=T),".bam"), 
      nthreads = 1)
```

### Activity
Perform the alignement for the same sample for both raw and trimmed data and check the stats.

## Counting
Now that we have figured out where each read comes from in the genome, we need to summarise the information across genes or exons. The alignment produces a set of BAM files, where each file contains the read alignments for each library. In the BAM file, there is a chromosomal location for every read that mapped uniquely. The mapped reads can be counted across grape genes by using the featureCounts <a href="#ref6">[6]</a> function from package `RSubread`. The code below uses the exon intervals defined in the NCBI gtf annotation the V1 grape genome. Reads that map to exons of genes are added together to obtain the count for each gene, with some care taken with reads that span exon-exon boundaries. featureCounts takes all the BAM files as input, and outputs an object which includes the count matrix. Each sample is a separate column, each row is a gene.

```{r}
gtf="/home/share/RNASeqPractical/references/ensemblgenomes_Vitis_vinifera_release-43/gtf/Vitis_vinifera.12X.43.chr.gtf"
bam <- "bam/raw"
bamFiles <- list.files(bam,pattern=".*bam$", full.names = TRUE)
fc <- featureCounts(files = bamFiles, 
    nthreads = 1,
    strandSpecific = 0,
    isPairedEnd = FALSE,
    GTF.featureType="gene",
    GTF.attrType="gene_id",
    annot.ext=gtf, 
    isGTFAnnotationFile=TRUE )
```

### Activity

- Take a look at the different slots of the featureCounts object (hint: names(fc))
- Check the alignment stats.
- Save the counts in a text file and write a external representation of the object in a file (see `?save`).

## EDA - Activity
- load the full counts matrix from `/home/share/RNASeqPractical/data/counts/subread_counts`; you can select either the trimmed fastq (`fc_trimmed.rda`) or the raw fastq (`fc_raw.rda`) for the forthcoming downstream analysis.
- From the counts matrix fc:
    - check the library size for all the samples 
    - show a barplot with library size information
    - do a pairwise comparison of samples using scatterplots (or the `smoothScatter()` function)

## EDA - barplot
```{r}
load("/home/share/RNASeqPractical/data/counts/subread_counts/fc_raw.rda")
#load("/home/share/RNASeqPractical/data/counts/subread_counts/fc_trimmed.rda")
# base R barplot
#barplot(colSums(fc$counts), las=2, border = 0, cex.lab=1, cex.axis=1, font=1,col.axis="black")
## ggplot2 barplot
options(repr.plot.width = 15, repr.plot.height = 15)
fc$counts %>% 
colSums() %>% 
tibble::enframe(name = NULL) %>% 
ggplot(aes(x=colnames(fc$counts),y=value)) + 
geom_col() + 
theme(axis.text.x = element_text(angle = 90, hjust = 1)) + 
xlab("Samples") + ylab("Successfully assigned alignments") + coord_flip()  + 
geom_label(aes(label=scales::comma(colSums(fc$counts))), nudge_y = 30000, size=2.5)
```

# Design
Retrieve metadata about the esperiment and samples. There are several ways to do this using R (e.g. package `GREP2`), the command line (`pysradb`) or directly from the web. My suggestion for this case is to go to [SRA Run Selector](https://www.ncbi.nlm.nih.gov/Traces/study/) and use either the SRA experiment id `SRP193991` or the Bioproject id `PRJNA539964` for downloading `RunInfo Table`; then you can read the downloaded file into R.

```{r}
SRP193991_metadata <- read.table("/home/share/RNASeqPractical/data/SraRunTable.txt", header=T, sep=",")
```

### Question
- Inspect the file
- What metadata tell us about the experiment?

# Build the target matrix
```{r}
targets <- SRP193991_metadata[,c("Run","sample_acc", "GEO_Accession", "Cultivar","developmental_stage","replicate")]
colnames(fc$counts) <- str_remove(colnames(fc$counts),".trim.bam")
```

# Sum over replicate samples
Before proceeding we need to condense the columns of the counts table so that counts are summed over technical replicate samples.

```{r}
dge <- edgeR::DGEList(fc$counts)
#dge <- edgeR::sumTechReps(dge, ID = targets$sample_acc)
dge <- edgeR::sumTechReps(dge, ID = targets$GEO_Accession)
targets <- targets[!duplicated(targets$GEO_Accession),]
```
### Activity
Check both the counts matrix and the targets dimensions before and after the summarization.

# EDA 

## PCA - 2D
The RNA samples can be clustered in two dimensions using Principal Component Analysis (PCA) plots <a href="#ref7">[7]</a> showing the first two principal components that explain the variability in the data using the regularized log count data.. This is both an analysis step and a quality control step to explore the overall differences between the expression profiles of the different samples.
```{r}
dds <- DESeqDataSetFromMatrix(
  countData = dge$counts,
  colData = targets,
  design = ~developmental_stage)
vsd <- vst(dds, blind=TRUE)
pcaData <- plotPCA(vsd, 
	intgroup=c("Cultivar","developmental_stage"), 
	returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=Cultivar, shape=developmental_stage)) +
  geom_point(size=4) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed() + ggtitle("vsd transformed counts: Cultivar and Developmental stage") +
  theme_bw()

```
## PCA - 3D
```{r}
tmp <- btools::plotPCA3D(vsd, intgroup = "developmental_stage", ntop = 500,  returnData = TRUE)
percentVar3D <- round(100 * attr(tmp, "percentVar"))
plot_ly(tmp, x = ~PC1, y = ~PC2, z = ~PC3, color = ~developmental_stage) %>%
  add_markers() %>%
  layout(scene = list(xaxis = list(title = paste0("PC1: ",percentVar3D[1],"% variance")),
                      yaxis = list(title = paste0("PC2: ",percentVar3D[2],"% variance")),
                      zaxis = list(title = paste0("PC3: ",percentVar3D[3],"% variance"))))
```
### Activity

Generate a PCA 3D showing point colored by Cultivar (hint: modify the `intgroup` parameter)

# Differential Expression Analysis

## Design Overview

```{r}
targets
```
- 3 cutivars (2 chinese and 1 italian)
- 3 developmental stages from 
	- Veraison:
		+ El-35: Berries begin to colour and enlarge
		+ EL-36: Berries with intermediate sugar values
	- Harvest
		+ EL-38: Berries havest-ripe

## Comparisons (Exercise)

Q: Which biological questions do you want to answer with this design?

### A. Between developmental_stage, same Cultivar

1. X-EL35vsX-EL36
2. X-EL35vsX-EL38
3. X-EL36vsX-EL38
4. Y-EL35vsy-EL36
5. Y-EL35vsy-EL38
6. Y-EL36vsy-EL38
7. Z-EL35vsZ-EL36
8. Z-EL35vsZ-EL38
9. Z-EL36vsZ-EL38

### B. Between Cultivar, same developmental_stage

1. X-EL35vsY-EL35
2. X-EL35vsZ-EL35
3. Y-EL35vsZ-EL35
4. X-EL36vsY-EL36 
5. X-EL36vsZ-EL36
6. Y-EL36vsZ-EL36
7. X-EL38vsY-EL38
8. X-EL38vsZ-EL38
9. Y-EL38vsZ-EL38


__All samples will be filtered and normalized together.__


## Design Group-Mean Parametrization

Linear modeling and differential expression analysis in limma requires a design matrix to be specified. The design matrix records which treatment conditions were applied to each samples, and it also defines how the experimental effects are parametrized in the linear models. The experimental design for this study can be viewed as a one-way layout and the design matrix can be constructed by:

```{r}
targets$group <- paste0(targets$Cultivar, ".", targets$developmental_stage)
f <-factor(targets$group,levels=unique(targets$group))
design <- model.matrix(~0+f)
colnames(design) <- unique(targets$group)
```

# Filtering
Genes that have very low counts across all the libraries should be removed prior to downstream analysis. This is justified on both biological and statistical grounds. From biological point of view, a gene must be expressed at some minimal level before it is likely to be translated into a protein or to be considered biologically important. From a statistical point of view, genes with consistently low counts are very unlikely be assessed as significantly DE because low counts do not provide enough statistical evidence for a reliable judgement to be made. Such genes can therefore be removed from the analysis without any loss of information. As a rule of thumb, we require that a gene have a count of at least 10 or so in at least some libraries before it is considered to be expressed in the study. In practice, the filtering is actually based on count-per-million (CPM) values so as to avoid favoring genes that are expressed in larger libraries over those expressed in smaller libraries. In edgeR, the filtering can be accomplished using a function that takes into account the library sizes and the experimental design:
```{r}
# The next step is to remove rows that consistently have zero or very low counts.
keep <- filterByExpr(dge, design)
table(keep)
dge <- dge[keep,]
```

# Normalization for composition bias

Normalization by trimmed mean of M values (TMM) is performed by using the calcNormFactors function, which returns the DGEList argument with only the norm.factors changed. It calculates a set of normalization factors, one for each sample, to eliminate composition biases between libraries. The product of these factors and the library sizes defines the effective library size, which replaces the original library size in all downstream analyses. The normalization factors of all the libraries multiply to unity. A normalization factor below one indicates that a small number of high count genes are monopolizing the sequencing, causing the counts for other genes to be lower than would be usual given the library size. As a result, the effective library size will be scaled down for that sample. Here we see, for example, tha GSM3737302, GSM3737303, GSM3737308 samples (there are more) have low normalization factors. This is a sign that these samples contain a number of very highly upregulated genes. Note. In general, we find TMM normalization to be satisfactory for almost all well-designed mRNA gene expression experiments. Single-cell RNA-seq is an exception, for which specialized normalization methods are needed (Lun, Bach, and Marioni 2016). Another, less common, type of study requiring special treatment is that with global differential expression, with more than half of the genome differentially expressed between experimental conditions in the same direction (Wu et al. 2013). Global differential expression should generally be avoided in well designed experiments. When it can’t be avoided, then some normalization reference such as spike-ins needs to be built into the experiment for reliable normalization to be done (Risso et al. 2014).

```{r}
dge <- calcNormFactors(dge, method = "TMM")
```

# Defining each treatment combination as a group
We now consider experiments with more than one experimental factor, but in which every
combination of experiment conditions can potentially have a unique effect. For example,
suppose that an experiment has been conducted on 3 different cultivars at three
developmental stages with all samples obtained from independent subjects. 
The data frame targets describes the treatment conditions applied to each sample.
Here we provide a contrast matrix for the contrast `development stage`:

## Contrast Matrices
```{r}
cont.matrix.EL <- makeContrasts(
  Xiangfei.EL35vsXiangfei.EL36=Xiangfei.EL35-Xiangfei.EL36, 
  Xiangfei.EL35vsXiangfei.EL38=Xiangfei.EL35-Xiangfei.EL38, 
  Xiangfei.EL36vsXiangfei.EL38=Xiangfei.EL36-Xiangfei.EL38, 
  Italia.EL35vsItalia.EL36=Italia.EL35-Italia.EL36,
  Italia.EL35vsItalia.EL38=Italia.EL35-Italia.EL38,
  Italia.EL36vsItalia.EL38=Italia.EL36-Italia.EL38,
  Zaomeiguixiang.EL35vsZaomeiguixiang.EL36=Zaomeiguixiang.EL35-Zaomeiguixiang.EL36,
  Zaomeiguixiang.EL35vsZaomeiguixiang.EL38=Zaomeiguixiang.EL35-Zaomeiguixiang.EL38,
  Zaomeiguixiang.EL36vsZaomeiguixiang.EL38=Zaomeiguixiang.EL36-Zaomeiguixiang.EL38,
  levels=design)
```
# Differential expression

## Linear modeling with limma-voom

We se voom() <a href="#ref10">[10]</a> to convert the read counts to log2-cpm, with associated weights, ready for linear modelling. The voom plot is used to check the mean-variance relationship of the expression data, after fitting a linear model. A scatterplot of residual-variances vs average log-expression is created. The plot is especially useful for examining the mean-variance trend.  It can be seen from this plot that the variance is no longer dependent on the mean expression level.
```{r}
cont.matrix <- cont.matrix.EL
v <- voom(dge, design, plot = TRUE)
fit_voom <- lmFit(v, design)
plotSA(fit_voom)
abline(a=1, b=0, col="blue")
fit_voom2 <- contrasts.fit(fit_voom, cont.matrix)
fit_voom2 <- eBayes(fit_voom2)
fit_voom2$Genes <- rownames(fc$counts)
```
## Developmental stages (EL) contrasts
```{r}
Xiangfei.EL35vsXiangfei.EL36 <- topTable(fit_voom2, coef=1,number = Inf, adjust.method = "BH", sort.by = "P")
Xiangfei.EL35vsXiangfei.EL36 %>% rownames_to_column() %>% arrange(-logFC,P.Value) %>% head()
fit_voom2_ELs_decideTest_separate <- summary(decideTests(fit_voom2, method="separate"))
nice.col <- brewer.pal(6,name="Dark2")
stripchart(v$E["VIT_07s0005g03420",]~targets$group,
	vertical=TRUE,las=2,cex.axis=0.8,pch=16,cex=1.3,col=nice.col,
	method="jitter",ylab="Normalised log2 expression",main="VIT_07s0005g03420")
```

## Cultivar contrasts - voom
```{r}
cont.matrix <- cont.matrix.cultivar
fit_voom2 <- contrasts.fit(fit_voom, cont.matrix)
fit_voom2 <- eBayes(fit_voom2)
fit_voom2$Genes <- rownames(fc$counts)
Xiangfei.EL35vsItalia.EL35 <- topTable(fit_voom2, coef=1,number = Inf, adjust.method = "BH", sort.by = "P")
Xiangfei.EL35vsItalia.EL35 %>% rownames_to_column() %>% arrange(-logFC,P.Value) %>% head()
fit_voom2_cutltivars_decideTest_separate <- summary(decideTests(fit_voom2, method="separate"))
```

# Differential expression: limma-trend
If the sequencing depth is reasonably consistent across the RNA samples, then the simplest and most
robust approach to differential exis to use limma-trend. This approach will usually work well if the
ratio of the largest library size to the smallest is not more than about 3-fold.
In the limma-trend approach, the counts are converted to logCPM values using edgeR’s cpm
function:
```{r}
logCPM <- cpm(dge, log=TRUE, prior.count=3)
```
__N.B.__ The prior count is used here to damp down the variances of logarithms of low counts.

The logCPM values can then be used in any standard limma pipeline, using the trend=TRUE argument when running eBayes or treat.

### Activity
- Create a contrast matrix ( named `cont.matrix.cultivar` ) which allows the selection of the differential expressed genes for all three cultivars in any stage, i.e.Xiangfei.EL35vsItalia.EL35, Xiangfei.EL35-Italia.EL35, Xiangfei.EL35vsZaomeiguixiang.EL35, etc..
- Use either the limma-voom or the following limma-trend procedure to retrieve a list of differential expressed genes for each contrast.

## Cultivar contrasts - trend 
```{r}
cont.matrix <- cont.matrix.cultivar
fit_voom <- lmFit(logCPM, design)
fit_voom2 <- contrasts.fit(fit_voom, cont.matrix)
fit_voom2 <- eBayes(fit_voom2, trend = TRUE)
fit_voom2$Genes <- rownames(fc$counts)
Xiangfei.EL35vsItalia.EL35 <- topTable(fit_voom2, coef=1,number = Inf, adjust.method = "BH", sort.by = "P")
Xiangfei.EL35vsItalia.EL35 %>% rownames_to_column() %>% arrange(-logFC,P.Value) %>% write_tsv("tT_Xiangfei.EL35vsItalia.EL35_trend.txt")
fit_voom2_cutltivars_decideTest_separate <- summary(decideTests(fit_voom2, method="separate"))

```

### plot of individual genes of interest
Before following up on the DE genes with further lab work, it is recommended to have a look at the expression levels of the individual samples for the genes of interest. We can quickly look at grouped expression using stripchart. We can use either the normalised log expression values in the voom object (`v$E`) or `logCPM`.

```{r}
nice.col <- brewer.pal(6,name="Dark2")
my_gene="VIT_11s0052g01650"
stripchart(v$E[my_gene,]~targets$group,
  vertical=TRUE,las=2,cex.axis=0.8,pch=16,cex=1.3,col=nice.col,
  method="jitter",ylab="Normalised log2 expression",main=my_gene)
```

#### Activity
Create a barplot showing  the number of up and down regulated genes in all contrasts.

#### Hint
Use the the table returned by `summary(decideTests(fit_voom2, method="separate"))` as data object and either the `barplot()` function or ggplot2 `geom_bar(stat="identity")` layer for build the plot.

## P-value distribution

```{r}
ggplot(Xiangfei.EL35vsItalia.EL35[!is.na(Xiangfei.EL35vsItalia.EL35$P.Value),], aes(x = P.Value)) +
    geom_histogram(alpha=.5, position='identity', bins = 50) +
    labs(title='Histogram of unadjusted p-values') +
    xlab('Unadjusted p-values') +
    xlim(c(0, 1.0005)) 
```

## Volcano plot

A volcano plot displays log fold changes on the x-axis versus a measure of statistical significance on the y-axis(Here $-log_{10}Pvalue$).

```{r}
results_voom <- decideTests(fit_voom2)
summary(decideTests(fit_voom2, lfc=1, p.value=0.05))
volcanoplot(fit_voom2, coef =1, highlight=20, names=rownames(fit_voom2), main="", pch=8, cex=0.35)
abline(h = -4.32, v = -1.5, col = "red", lty = 3); abline(h = 4.32, v = 1.5, col = "red", lty = 3)

```

## Heat map
Heatmaps are a way to display differential expression results. To create a heatmap, we  select the first n DE genes log2-counts-per-million (logCPM) values.
```{r}
tT <- Xiangfei.EL35vsItalia.EL35
o <- order(tT$P.Value)
logCPM_o <- logCPM[o[1:30],]
```

A heat map can then be produced by the coolmap function which clusters genes by correlation (highly correlated genes are closest) and clusters samples based on Euclidean distance between the expression values.
```{r}
coolmap(logCPM_o, margins=c(7,7), lhei=c(1,6), lwid=c(1,3))
```

# Functional Analysis (Pathway analysis)

## Gene ontology analysis using GOseq 

GOseq allows to perform a Gene Ontology (GO) analysis tailored to RNA-seq data as it accounts for the gene length bias in detection of over-representation  <a href="#ref12">[12]</a>

From the [GOseq vignette](https://www.bioconductor.org/packages/release/bioc/vignettes/goseq/inst/doc/goseq.pdf):

- GOseq first needs to quantify the length bias present in the dataset under consideration.
- This is done by calculating a Probability Weighting Function or PWF which can be thought of as a function which gives the probability that a gene will be differentially expressed (DE), based on its length alone.
- The PWF is calculated by fitting a monotonic spline to the binary data series of differential expression (1=DE, 0=Not DE) as a function of gene length.
- The PWF is used to weight the chance of selecting each gene when forming a null distribution for GO category membership.
- The fact that the PWF is calculated directly from the dataset under consideration makes this approach robust, only correcting for the length bias present in the data.
>“GO analysis of RNA-seq data requires the use of random sampling in order to generate a suitable null distribution for GO category membership and calculate each category’s significance for over representation amongst DE genes. … In most cases, the Wallenius distribution can be used to approximate the true null distribution, without any significant loss in accuracy. The goseq package implements this approximation as its default option.”

In order to perform the analysis two vectors are required: one containing all genes assayed in the RNA-seq experiment, the other containing all genes which are DE. If we assume that the vector of genes being assayed is namedassayed.genes and the vector of DE genes is named de.genes we can construct a named vector
suitable for use with goseq using the following:

```{r}
library(goseq)
assayed.genes <- fc$annotation$GeneID
de.genes <-  Xiangfei.EL35vsXiangfei.EL36[(Xiangfei.EL35vsXiangfei.EL36$logFC > 1) & Xiangfei.EL35vsXiangfei.EL36$adj.P.Val< 0.01 ,]
gene.vector <- as.integer(assayed.genes%in%rownames(de.genes))
names(gene.vector) <- assayed.genes
```

### Activity
Replace the select contrast with one of your interest end change the double cut-off for the selection of DE genes for the enrichment analysis

## Non-native Gene Identifier or category test
If the organism, Gene Identifier or category test you wish to perform is not in the native goseq
database, you will have to supply one or all of the following:
- Length data: the length of each gene in your gene identifier format.
- Category mappings: the mapping (usually many-to-many) between the categories you wish to test for over/under representation amongst DE genes and genes in your gene identifier format.
```{r}
GO_ann=read.table("/home/share/RNASeqPractical/references/GO_plant_slim_fixed.tab",header=F,sep="\t")[,1:2]
colnames(GO_ann) <- c("GeneIDs","GOTerms") 
```

Note: GO annotation is retrieved from [here](http://genomes.cribi.unipd.it/DATA/V1/ANNOTATION/).

## GO Analysis

### Fitting the Probability Weighting Function (PWF)
We first need to obtain a weighting for each gene, depending on its length, given by the PWF.
As you may have noticed when running supportedGenomes or supportedGeneIDs, length data is
available in the local database for our gene ID, “ensGene” and our genome, “hg19”. We will let
goseq automatically fetch this data from its databases.
```{r}
pwf=nullp(gene.vector,bias.data=fc$annotation$Length)
```
### Using the Wallenius approximation
To start with we will use the default method, to calculate the over and under expressed GO
categories among DE genes.
```{r}
GO.wall=goseq(pwf,gene2cat=GO_ann)
head(GO.wall)
```
### Making sense of the results
If you wish to identify categories significantly enriched/unenriched below some p-value cutoff, it is recommended to
first apply some kind of multiple hypothesis testing correction. For example, GO categories over
enriched using a .05 FDR cutoff are:
```{r}
enriched.GO=GO.wall$category[p.adjust(GO.wall$over_represented_pvalue, method="BH")<.05]
```
Information about each term can be obtained from the [Gene Ontology website](http://www.geneontology.org) or tool like [http://revigo.irb.hr/](http://revigo.irb.hr/) or [http://bioinfo.cau.edu.cn/agriGO/](http://bioinfo.cau.edu.cn/agriGO/) or in R:
```{r}
library(GO.db)
for(go in enriched.GO){
print(GOTERM[[go]])
cat("--------------------------------------\n")
}
```
### Plot top 10 GOTERMS <a href="#ref13">[13]</a> 

```{r}
GO.wall %>% 
    top_n(10, wt=-over_represented_pvalue) %>% 
    mutate(hitsPerc=numDEInCat*100/numInCat) %>% 
    ggplot(aes(x=hitsPerc, 
               y= paste0(category," - ", term), 
               colour=over_represented_pvalue, 
               size=numDEInCat)) +
        geom_point() +
        expand_limits(x=0) +
        labs(x="Hits (%)", y="GO term", colour="p value", size="Count")
```

# Packages used
This practical depends on various packages from version 3.10 of the Bioconductor project, running on R version 3.6.2 or higher. The complete list of the packages used for this workflow are shown below:

In [34]:
(my_session <- sessionInfo())
#save.image(paste0("subread_analysis_",format(Sys.Date(), "%d-%m-%Y"),".RData"), compress = TRUE)

R version 3.6.2 (2019-12-12)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Debian GNU/Linux 10 (buster)

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.8.0
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.8.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] parallel  stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] GO.db_3.10.0                AnnotationDbi_1.48.0       
 [3] goseq_1.38.0                geneLenDataBase_1.22.0     
 [5] BiasedUrn_1.07              edgeR_3.28.0               
 [7] limma_3.42.0                DESeq2_1.26.0              
 [9] SummarizedExperiment_1.16.1 DelayedArray_0.

# References

<a id='ref1'></a>1. Sun,L., Zhu,B., Zhang,X., Zhang,G., Yan,A., Wang,H., Wang,X. and Xu,H. (2019) Transcriptome profiles of three Muscat table grape cultivars to dissect the mechanism of terpene biosynthesis. Sci Data, 6.

<a id='ref2'></a>2. [Mapping between SRA IDs and accessing metadata with the Bioconductor SRAdb package](http://bergmanlab.genetics.uga.edu/?p=1072)

<a id='ref2bis'></a>2bis. Choudhary, S. (2019) pysradb: A Python package to query next-generation sequencing metadata and data from NCBI Sequence Read Archive. F1000Res, 8, 532.

<a id='ref3'></a>3. Babraham Bioinformatics - FastQC A Quality Control tool for High Throughput Sequence Data. Babraham Bioinformatics - FastQC A Quality Control tool for High Throughput Sequence Data 
https://www.bioinformatics.babraham.ac.uk/projects/fastqc/ (accessed Nov 21, 2019).

<a id='ref4'></a>4. Bolger, A. M., Lohse, M. and Usadel, B. (2014) Trimmomatic: a flexible trimmer for Illumina sequence data. Bioinformatics, 30, 2114–2120.

<a id='ref5'></a>5. Y. Liao, G. K. Smyth and W. Shi. “The Subread aligner: fast, accurate and scalable read mapping by seed-and-vote”. In: Nucleic Acids Research 41 (2013), p. 108.

<a id='ref6'></a>6. Liao, Y., Smyth, G. K. and Shi, W. (2014) featureCounts: an efficient general purpose program for assigning sequence reads to genomic features. Bioinformatics, 30, 923–930.

<a id='ref7'></a>7. [Principal Component Analysis Explained Visually](http://setosa.io/ev/principal-component-analysis/)

<a id='ref8'></a>8. Robinson, M. D., and A. Oshlack. 2010. “A Scaling Normalization Method for Differential Expression Analysis of RNA-seq Data.” Genome Biology 11 (3):R25.

<a id='ref9'></a>9. Chen, Y., Lun, A. T. L. and Smyth, G. K. (2016) From reads to genes to pathways: differential expression analysis of RNA-Seq experiments using Rsubread and the edgeR quasi-likelihood pipeline. F1000Research, 5, 1438.

<a id='ref10'></a>10. Law, C. W., Chen, Y., Shi, W., et al. (2014) voom: precision weights unlock linear model analysis tools for RNA-seq read counts. Genome Biology, 15, R29.

<a id='ref11'></a>11. Ritchie, M. E., Phipson, B., Wu, D., et al. (2015) limma powers differential expression analyses for RNA-sequencing and microarray studies. Nucl. Acids Res., gkv007.

<a id='ref12'></a>12. Young, M. D., Wakefield, M. J., Smyth, G. K., et al. (2010) Gene ontology analysis for RNA-seq: accounting for selection bias. Genome Biology, 11, R14.

<a id='ref13'></a>13. [GO enrichment analysis](https://bioinformatics-core-shared-training.github.io/cruk-summer-school-2018/RNASeq2018/html/06_Gene_set_testing.nb.html#go-enrichment-analysis)